Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [1]:
import numpy as np
import requests

API_KEY = 'ei_a4a199d2c331dd57709baf622fca6b83528f34504943bad629b613d38a1e4b17'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/3973/training/7/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/3973/training/7/y', headers={'x-api-key': API_KEY})).content

Store the data in a temporary file, and load it back through Numpy.

In [2]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [3]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_random_seed(RANDOM_SEED)

classes_values = [ "cough", "no_cough" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    shuffle_buffer_size = batch_size * 3
    train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(batch_size, drop_remainder=True).prefetch(1)
    validation_dataset = validation_dataset.shuffle(shuffle_buffer_size).batch(batch_size, drop_remainder=True).prefetch(1)
    return train_dataset, validation_dataset

callbacks = []


C:\Users\digvi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\digvi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\digvi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\digvi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Train the model:

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

# model architecture
model = Sequential()
model.add(InputLayer(input_shape=(input_length, ), name='x_input'))
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(30, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5, padding='same'))
model.add(Conv1D(10, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5, padding='same'))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)

# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 4
train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_dataset, epochs=100, validation_data=validation_dataset, verbose=2, callbacks=callbacks)


Epoch 1/100
115/115 - 0s - loss: 0.8376 - acc: 0.3935 - val_loss: 0.7601 - val_acc: 0.3621
Epoch 2/100
115/115 - 0s - loss: 0.7254 - acc: 0.4239 - val_loss: 0.7118 - val_acc: 0.4483
Epoch 3/100
115/115 - 0s - loss: 0.6847 - acc: 0.4848 - val_loss: 0.6912 - val_acc: 0.4741
Epoch 4/100
115/115 - 0s - loss: 0.6639 - acc: 0.5261 - val_loss: 0.6778 - val_acc: 0.6207
Epoch 5/100
115/115 - 0s - loss: 0.6476 - acc: 0.6696 - val_loss: 0.6665 - val_acc: 0.6552
Epoch 6/100
115/115 - 0s - loss: 0.6327 - acc: 0.6913 - val_loss: 0.6574 - val_acc: 0.6638
Epoch 7/100
115/115 - 0s - loss: 0.6193 - acc: 0.7065 - val_loss: 0.6490 - val_acc: 0.6638
Epoch 8/100
115/115 - 0s - loss: 0.6065 - acc: 0.7174 - val_loss: 0.6421 - val_acc: 0.6552
Epoch 9/100
115/115 - 0s - loss: 0.5943 - acc: 0.7326 - val_loss: 0.6365 - val_acc: 0.6552
Epoch 10/100
115/115 - 0s - loss: 0.5825 - acc: 0.7370 - val_loss: 0.6309 - val_acc: 0.6638
Epoch 11/100
115/115 - 0s - loss: 0.5710 - acc: 0.7543 - val_loss: 0.6253 - val_acc: 0.65

In [5]:
# Save the model to disk
model.save('saved_model')